# Enhanced SQL Database Management

This notebook demonstrates the enhanced SQL features with pre-configured database connections and SQL templates.

In [ ]:
# Load the SQL configuration
import sys
sys.path.append('/home/jovyan/sql-configs')

from sql_connections_template import (
    DATABASE_CONNECTIONS, 
    SQL_TEMPLATES,
    list_connections,
    get_database_engine,
    get_sql_template
)

## 1. View Available Database Connections

In [ ]:
# Show all pre-configured database connections
list_connections()

## 2. Connection Selection Dropdown

Select your database connection from the dropdown:

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create dropdown for database selection
db_dropdown = widgets.Dropdown(
    options=[(config['name'], key) for key, config in DATABASE_CONNECTIONS.items()],
    description='Database:',
    style={'description_width': 'initial'}
)

display(db_dropdown)

## 3. SQL Template Dropdown

In [ ]:
# Create dropdown for SQL templates
sql_dropdown = widgets.Dropdown(
    options=list(SQL_TEMPLATES.keys()),
    description='SQL Template:',
    style={'description_width': 'initial'}
)

# Create text inputs for template parameters
table_name_input = widgets.Text(
    value='your_table_name',
    description='Table Name:',
    style={'description_width': 'initial'}
)

column_name_input = widgets.Text(
    value='your_column_name',
    description='Column Name:',
    style={'description_width': 'initial'}
)

display(sql_dropdown, table_name_input, column_name_input)

## 4. Generate SQL Query

In [ ]:
# Generate SQL based on selections
def generate_sql():
    template_name = sql_dropdown.value
    table_name = table_name_input.value
    column_name = column_name_input.value
    
    try:
        query = get_sql_template(
            template_name,
            table_name=table_name,
            column_name=column_name,
            date_column=column_name
        )
        print("Generated SQL:")
        print("=" * 50)
        print(query)
        return query
    except Exception as e:
        print(f"Error generating SQL: {e}")
        return None

# Button to generate SQL
generate_button = widgets.Button(description="Generate SQL")
generate_button.on_click(lambda x: generate_sql())

display(generate_button)

## 5. Connect to Selected Database

In [ ]:
# Load SQL extension
%load_ext sql

# Connect to selected database
selected_db = db_dropdown.value
connection_string = DATABASE_CONNECTIONS[selected_db]['connection_string']

print(f"Connecting to: {DATABASE_CONNECTIONS[selected_db]['name']}")
print(f"Description: {DATABASE_CONNECTIONS[selected_db]['description']}")

# Note: Update the connection string with real credentials before using
%sql $connection_string

## 6. Execute SQL Queries

Now you can run SQL queries using the `%%sql` magic command:

In [ ]:
%%sql
-- Example: List all tables (works for most databases)
SELECT table_name FROM information_schema.tables 
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
LIMIT 10;

## 7. Using JupySQL for Advanced Features

In [ ]:
# JupySQL provides enhanced SQL capabilities
import jupysql

# Enable JupySQL
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

print("JupySQL enabled with pandas integration!")

## 8. Quick Connection Switcher

In [ ]:
# Quick function to switch between databases
def switch_database(db_key):
    if db_key in DATABASE_CONNECTIONS:
        config = DATABASE_CONNECTIONS[db_key]
        print(f"Switching to: {config['name']}")
        %sql $config['connection_string']
    else:
        print(f"Unknown database: {db_key}")
        print(f"Available: {list(DATABASE_CONNECTIONS.keys())}")

# Example usage:
# switch_database('duckdb_memory')
# switch_database('local_postgres')